# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---


# <h1><center>Clase 05: Acceso a datos en internet con `requests`</center></h1>

**Objetivo:** mostrar, con ejemplos prácticos, cómo usar la librería `requests` para obtener datos desde la web (JSON, CSV, binarios), manejar parámetros, errores y guardar los resultados para su análisis con `pandas`.


## 1. Setup

Instalaremos/Importaremos las librerías necesarias. En equipos con Anaconda, ya deberían estar disponibles.


In [1]:
import requests
import pandas as pd
from pathlib import Path
from time import sleep
print("Librerías listas.")

Librerías listas.


In [3]:
import requests
import pandas as pd
from pathlib import Path
from time import sleep
print("La librerias están listas")

La librerias están listas



## 2. Descarga de **CSV** desde una URL directa (raw.githubusercontent.com)

Usaremos un dataset público del repositorio **FiveThirtyEight** (college majors).**URL:** `https://raw.githubusercontent.com/fivethirtyeight/data/master/college-majors/recent-grads.csv`

Demostraremos:
- Descarga con `requests.get`
- Guardado local en `data/`
- Carga con `pandas.read_csv`
- Verificación del tamaño descargado


In [2]:
# URL del CSV con los datos
csv_url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/college-majors/recent-grads.csv"
# lugar donde guardarlo en el computador local
out_dir = Path("data")   #donde lo quiero guardar
out_dir.mkdir(exist_ok=True)
csv_path = out_dir / "recent-grads.csv"

# solicitar los datos
r = requests.get(csv_url, timeout=30)
r.raise_for_status()  # Lanza error si el status no es 200
csv_path.write_bytes(r.content)

print("Guardado en:", csv_path.resolve())
print("Tamaño (bytes):", csv_path.stat().st_size)

Guardado en: C:\Users\const\OneDrive\Documentos\Clase IMT2200-25s2\clases\clase 05\data\recent-grads.csv
Tamaño (bytes): 26872


In [3]:
df_csv = pd.read_csv(csv_path)
df_csv.head()

,Rank,Major_code,Major,Total,Men,Women,Major_category,ShareWomen,Sample_size,Employed,...,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
0,1,2419,PETROLEUM ENGINEERING,2339.0,2057.0,282.0,Engineering,0.120564,36,1976,...,270,1207,37,0.018381,110000,95000,125000,1534,364,193
1,2,2416,MINING AND MINERAL ENGINEERING,756.0,679.0,77.0,Engineering,0.101852,7,640,...,170,388,85,0.117241,75000,55000,90000,350,257,50
2,3,2415,METALLURGICAL ENGINEERING,856.0,725.0,131.0,Engineering,0.153037,3,648,...,133,340,16,0.024096,73000,50000,105000,456,176,0
3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,1258.0,1123.0,135.0,Engineering,0.107313,16,758,...,150,692,40,0.050125,70000,43000,80000,529,102,0
4,5,2405,CHEMICAL ENGINEERING,32260.0,21239.0,11021.0,Engineering,0.341631,289,25694,...,5180,16697,1672,0.061098,65000,50000,75000,18314,4440,972



## 3. Manejo de **errores** y **timeouts**

Buenas prácticas:
- Usar `timeout=` para evitar que el programa quede colgado.
- Verificar `response.status_code` o usar `raise_for_status()`.
- Capturar excepciones con `try/except` y registrar mensajes útiles.


In [4]:
bad_url = "https://api.github.com/this/endpoint/does/not/exist"

try:
    r = requests.get(bad_url, timeout=5)
    r.raise_for_status()
except requests.exceptions.HTTPError as e:
    print("HTTPError:", e)
except requests.exceptions.Timeout:
    print("Timeout alcanzado.")
except requests.exceptions.RequestException as e:
    print("Error de red:", e)

HTTPError: 404 Client Error: Not Found for url: https://api.github.com/this/endpoint/does/not/exist



## 4. Sesiones y cabeceras personalizadas (`Session`), backoff simple

A veces conviene reutilizar conexiones con `requests.Session()` y establecer **cabeceras** (headers) comunes. También podemos implementar un **reintento simple** (backoff) frente a códigos temporales (p. ej., 429 o 5xx).


In [5]:
session = requests.Session()
session.headers.update({
    "Accept": "application/vnd.github+json",
    "User-Agent": "imt2200-class-notebook"
})

def get_with_retry(url, params=None, max_retries=3, backoff=2):
    for i in range(1, max_retries+1):
        resp = session.get(url, params=params, timeout=15)
        if resp.status_code == 200:
            return resp
        print(f"Intento {i}: status {resp.status_code}. Reintentando en {backoff} s...")
        sleep(backoff)
    resp.raise_for_status()

# Prueba reutilizando la API de GitHub (misma consulta de antes)
resp_ok = get_with_retry("https://api.github.com/search/repositories", params={
    "q": "geopandas language:python",
    "sort": "stars",
    "order": "desc",
    "per_page": 5
})
pd.json_normalize(resp_ok.json().get("items", []))[["full_name","stargazers_count","html_url"]].head()

,full_name,stargazers_count,html_url
0,geopandas/geopandas,4858,https://github.com/geopandas/geopandas
1,movingpandas/movingpandas,1327,https://github.com/movingpandas/movingpandas
2,ResidentMario/geoplot,1175,https://github.com/ResidentMario/geoplot
3,PatrikHlobil/Pandas-Bokeh,888,https://github.com/PatrikHlobil/Pandas-Bokeh
4,developmentseed/lonboard,811,https://github.com/developmentseed/lonboard



## 5. De **JSON anidado** a **tabla** (normalización)

Cuando la respuesta incluye estructuras anidadas, `pandas.json_normalize` ayuda a "aplanar" el JSON.


In [6]:
headers = {
    "Accept": "application/vnd.github+json",
    "User-Agent": "imt2200-class-notebook"
}
resp = requests.get("https://api.github.com/repos/pandas-dev/pandas/issues",
                    params={"state":"open", "per_page":10},
                    headers=headers, timeout=15)

In [7]:
issues = resp.json()
issues_df = pd.json_normalize(issues, sep="_", max_level=2)[["number","title","state","user_login","comments","created_at","updated_at","html_url"]]
issues_df.head()

,number,title,state,user_login,comments,created_at,updated_at,html_url
0,62143,DOC: Clarify is_scalar docstring to specify sc...,open,Aniketsy,0,2025-08-19T07:22:18Z,2025-08-19T07:22:18Z,https://github.com/pandas-dev/pandas/pull/62143
1,62142,DEPR: Categorical with values not present in c...,open,jbrockmendel,0,2025-08-18T21:30:22Z,2025-08-19T02:48:17Z,https://github.com/pandas-dev/pandas/pull/62142
2,62141,ENH: Consistent `name` property for the iterat...,open,FelixBenning,2,2025-08-18T10:00:32Z,2025-08-19T05:53:28Z,https://github.com/pandas-dev/pandas/issues/62141
3,62139,BUG: Fix to_csv microsecond inconsistency (#62...,open,prazian,0,2025-08-18T07:58:24Z,2025-08-18T14:05:08Z,https://github.com/pandas-dev/pandas/pull/62139
4,62138,BUG: Preserve day freq on DatetimeIndex subtra...,open,23f-1000003,1,2025-08-18T06:05:44Z,2025-08-18T06:06:44Z,https://github.com/pandas-dev/pandas/pull/62138



## 6. Guardar resultados procesados

Exportamos a CSV/Parquet para reutilizar los datos luego, sin necesidad de volver a llamar a la API.


In [8]:
# lugar donde almacenar los datos
proc_dir = Path("outputs")
proc_dir.mkdir(exist_ok=True)

# grabar
issues_df.to_csv(proc_dir / "pandas_issues_sample.csv", index=False)
print("Archivos exportados en:", proc_dir.resolve())

Archivos exportados en: C:\Users\const\OneDrive\Documentos\Clase IMT2200-25s2\clases\clase 05\outputs



---

### 📌 Buenas prácticas y ética de uso

- **Respeta** los términos de uso de las APIs y sitios web (TOS).  No hagas scraping de sitios que lo prohíben o requieran autenticación.  
- Revisa **límites de rate limit** (p. ej., GitHub limita requests por minuto sin token).  
- **Cachea** o guarda resultados para evitar llamadas innecesarias.  
- **Identifícate** con un `User-Agent` claro y usa `timeout` y reintentos responsables.  
- Cita las **fuentes** y documenta el **proceso** en tu repositorio.

---
